# Load and Set up Binary Classification

In [ ]:
!pip install -q datasets scikit-learn tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 6.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.metrics import classification_report
from datasets import load_dataset
from tqdm import tqdm

# Load IMDb dataset from HuggingFace
imdb = load_dataset("imdb")
train_data = imdb["train"]
test_data = imdb["test"]

train_texts = train_data["text"]
train_labels = train_data["label"]
test_texts = test_data["text"]
test_labels = test_data["label"]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
# Tokenization
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)

X_train_seq = tokenizer.texts_to_sequences(train_texts)
X_test_seq = tokenizer.texts_to_sequences(test_texts)

# Padding
max_len = 300
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post', truncating='post')

y_train = np.array(train_labels)
y_test = np.array(test_labels)

# LSTM-Model

In [ ]:
def build_binary_lstm_model():
    model = Sequential([
        Embedding(input_dim=10000, output_dim=128, input_length=max_len),
        LSTM(64, dropout=0.2, recurrent_dropout=0.2),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [ ]:
#Train and Evalyuate LSTM Model
lstm_model = build_binary_lstm_model()
lstm_model.summary()

lstm_model.fit(X_train_pad, y_train, epochs=3, batch_size=128, validation_split=0.1)
lstm_preds = (lstm_model.predict(X_test_pad) > 0.5).astype("int32")
# Save LSTM
lstm_model.save("lstm_binary_model.h5")

print("LSTM Classification Report:")
print(classification_report(y_test, lstm_preds))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/3
176/176 ━━━━━━━━━━━━━━━━━━━━ 166s 886ms/step - accuracy: 0.5504 - loss: 0.6883 - val_accuracy: 0.0020 - val_loss: 0.7860
Epoch 2/3
176/176 ━━━━━━━━━━━━━━━━━━━━ 122s 692ms/step - accuracy: 0.5756 - loss: 0.6775 - val_accuracy: 0.1388 - val_loss: 0.8274
Epoch 3/3
176/176 ━━━━━━━━━━━━━━━━━━━━ 126s 718ms/step - accuracy: 0.6279 - loss: 0.6247 - val_accuracy: 0.1312 - val_loss: 0.7977
782/782 ━━━━━━━━━━━━━━━━━━━━ 63s 80ms/step


LSTM Classification Report:
              precision    recall  f1-score   support

           0       0.52      0.94      0.67     12500
           1       0.71      0.14      0.23     12500

    accuracy                           0.54     25000
   macro avg       0.62      0.54      0.45     25000
weighted avg       0.62      0.54      0.45     25000



# Convolutional Neural Network

In [ ]:
def build_binary_cnn_model():
    model = Sequential([
        Embedding(input_dim=10000, output_dim=128, input_length=max_len),
        Conv1D(128, 5, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [ ]:
#Train and evaluate CNN Model
cnn_model = build_binary_cnn_model()
cnn_model.summary()

cnn_model.fit(X_train_pad, y_train, epochs=3, batch_size=128, validation_split=0.1)
cnn_preds = (cnn_model.predict(X_test_pad) > 0.5).astype("int32")
#Save CNN Model
cnn_model.save("cnn_binary_model.h5")

print("CNN Classification Report:")
print(classification_report(y_test, cnn_preds))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/3
176/176 ━━━━━━━━━━━━━━━━━━━━ 85s 475ms/step - accuracy: 0.6182 - loss: 0.6312 - val_accuracy: 0.7656 - val_loss: 0.5193
Epoch 2/3
176/176 ━━━━━━━━━━━━━━━━━━━━ 87s 490ms/step - accuracy: 0.8857 - loss: 0.2852 - val_accuracy: 0.8816 - val_loss: 0.2976
Epoch 3/3
176/176 ━━━━━━━━━━━━━━━━━━━━ 138s 471ms/step - accuracy: 0.9599 - loss: 0.1340 - val_accuracy: 0.7896 - val_loss: 0.5234
782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 34ms/step


CNN Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.92      0.88     12500
           1       0.91      0.82      0.86     12500

    accuracy                           0.87     25000
   macro avg       0.88      0.87      0.87     25000
weighted avg       0.88      0.87      0.87     25000



#Transformer Model

In [ ]:
from transformers import DistilBertTokenizerFast, TFDistilBertForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import Trainer, TrainingArguments
import tensorflow as tf

# Load tokenizer and model
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
transformer_model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
# Tokenization function
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding=True, max_length=512)

# Tokenize datasets
encoded_imdb = imdb.map(tokenize_function, batched=True)
encoded_imdb = encoded_imdb.rename_column("label", "labels")
encoded_imdb.set_format(type="tensorflow", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
# Convert HuggingFace dataset to TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        "input_ids": encoded_imdb["train"]["input_ids"],
        "attention_mask": encoded_imdb["train"]["attention_mask"]
    },
    encoded_imdb["train"]["labels"]
)).shuffle(1000).batch(16)

test_dataset = tf.data.Dataset.from_tensor_slices((
    {
        "input_ids": encoded_imdb["test"]["input_ids"],
        "attention_mask": encoded_imdb["test"]["attention_mask"]
    },
    encoded_imdb["test"]["labels"]
)).batch(16)


In [ ]:
from transformers import create_optimizer

# Total training steps (adjust for your case)
batch_size = 16
num_epochs = 3
num_train_steps = len(train_dataset) * num_epochs
optimizer, schedule = create_optimizer(
    init_lr=5e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps
)

transformer_model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

# Now train
transformer_model.fit(train_dataset, validation_data=test_dataset, epochs=num_epochs)

Epoch 1/3


In [ ]:
# Get predictions
logits = transformer_model.predict(test_dataset).logits
preds = tf.argmax(logits, axis=1).numpy()

# True labels
true_labels = np.array(encoded_imdb["test"]["labels"])

# Evaluation report
from sklearn.metrics import classification_report
print("DistilBERT Classification Report:")
print(classification_report(true_labels, preds))